# Python for Data Analysis v2 | Notes_ Chapter_4 NumPy 基础：数组和矢量计算

>学而时习之，不亦说乎

本人以简书作者 SeanCheney 系列专题文章并结合原书为学习资源，记录个人笔记，仅作为知识记录及后期复习所用，原作者地址查看 [简书 SeanCheney](https://www.jianshu.com/u/130f76596b02)，如有错误，还望批评指教。——ZJ


>原作者：SeanCheney | [《利用Python进行数据分析·第2版》第4章 NumPy基础：数组和矢量计算](https://www.jianshu.com/p/a380222a3292) | 來源：简书

>[Github:wesm](https://github.com/wesm/pydata-book) | [Github:中文 BrambleXu](https://github.com/BrambleXu/pydata-notebook)|
简书:[利用Python进行数据分析·第2版](https://www.jianshu.com/c/52882df3377a)

环境：Python 3.6 

---

## 第 4 章 NumPy 基础：数组和矢量计算


NumPy（Numerical Python 的简称）是 Python数 值计算最重要的基础包。大多数提供科学计算的包都是用 NumPy 的数组作为构建基础。

NumPy 的部分功能如下：

- ndarray，一个具有矢量算术运算和复杂广播能力的快速且节省空间的多维数组。
- 用于对整组数据进行快速运算的标准数学函数（无需编写循环）。
- 用于读写磁盘数据的工具以及用于操作内存映射文件的工具。
- 线性代数、随机数生成以及傅里叶变换功能。
- 用于集成由 C、C++、Fortran 等语言编写的代码的 A C API。


对于大部分数据分析应用而言，我最关注的功能主要集中在：

- 用于数据整理和清理、子集构造和过滤、转换等快速的矢量化数组运算。
- 常用的数组算法，如排序、唯一化、集合运算等。
- 高效的描述统计和数据聚合/摘要运算。
- 用于异构数据集的合并/连接运算的数据对齐和关系型数据运算。
- 将条件逻辑表述为数组表达式（而不是带有if-elif-else分支的循环）。
- 数据的分组运算（聚合、转换、函数应用等）

NumPy 之于数值计算特别重要的原因之一，是因为它可以高效处理大数组的数据。这是因为：

- NumPy 是在一个连续的内存块中存储数据，独立于其他 Python 内置对象。NumPy 的 C 语言编写的算法库可以操作内存，而不必进行类型检查或其它前期工作。比起 Python 的内置序列，NumPy 数组使用的内存更少。
- NumPy 可以在整个数组上执行复杂的计算，而不需要 Python 的 for 循环。

要搞明白具体的性能差距，考察一个包含一百万整数的数组，和一个等价的Python 列表:

```
In [1]: import numpy as np

In [2]: my_arr = np.arange(1000000)

In [3]: my_list = list(range(1000000))

# 各个序列分别乘以 2：

In [4]: %time for _ in range(10):my_arr2 = my_arr * 2
Wall time: 46.9 ms

In [5]: %time for _ in range(10): my_list = [x * 2 for x in my_list]
Wall time: 913 ms


```

基于 NumPy 的算法要比纯 Python 快 10 到 100 倍（甚至更快），并且使用的内存更少。

### 4.1 NumPy 的 ndarray：一种多维数组对象

NumPy 最重要的一个特点就是其 N 维数组对象（即 ndarray），该对象是一个快速而灵活的大数据集容器。

Python 是如何利用与标量值类似的语法进行批次计算:

```
In [6]: data = np.random.randn(2,3)

In [7]: data
Out[7]:
array([[-0.7949888 ,  0.59269599,  0.6047145 ],
       [-0.05579778, -1.97623901, -0.52438662]])

In [8]: data * 10
Out[8]:
array([[ -7.94988799,   5.92695992,   6.04714496],
       [ -0.55797776, -19.76239014,  -5.24386616]])

In [9]: data + data
Out[9]:
array([[-1.5899776 ,  1.18539198,  1.20942899],
       [-0.11159555, -3.95247803, -1.04877323]])


```

ndarray 是一个通用的同构数据多维容器，也就是说，其中的所有元素必须是相同类型的。每个数组都有一个 shape（一个表示各维度大小的元组）和一个dtype（一个用于说明数组数据类型的对象）：


```
In [10]: data.shape
Out[10]: (2, 3)

In [11]: data.dtype
Out[11]: dtype('float64')


```

- 笔记：当你在本书中看到“数组”、“NumPy 数组”、"ndarray"时，基本上都指的是同一样东西，即 ndarray 对象。


####  创建 ndarray

创建数组最简单的办法就是使用 array 函数。它接受一切序列型的对象（包括其他数组），然后产生一个新的含有传入数据的 NumPy 数组。以一个列表的转换为例：

```
In [12]: data1 = [6, 7.5, 8, 0, 1]

In [13]: arr1 = np.array(data1)

In [14]: arr1
Out[14]: array([6. , 7.5, 8. , 0. , 1. ])

# 嵌套序列（比如由一组等长列表组成的列表）将会被转换为一个多维数组：

In [15]: data2 = [[1,2,3,4],[5,6,7,8]]

In [17]: arr2 = np.array(data2)

In [18]: arr2
Out[18]:
array([[1, 2, 3, 4],
       [5, 6, 7, 8]])


```
因为 data2 是列表的列表，NumPy 数组 arr2 的两个维度的 shape 是从 data2引入的。可以用属性 ndim 和 shape 验证：

```
In [19]: arr2.ndim
Out[19]: 2

In [20]: arr2.shape
Out[20]: (2, 4)


```

除非特别说明（稍后将会详细介绍），np.array 会尝试为新建的这个数组推断出一个较为合适的数据类型。数据类型保存在一个特殊的dtype对象中。比如说，在上面的两个例子中，我们有：

```
In [21]: arr1.dtype
Out[21]: dtype('float64')

In [22]: arr2.dtype
Out[22]: dtype('int32')


```

除 np.array 之外，还有一些函数也可以新建数组。比如，zeros 和 ones 分别可以创建指定长度或形状的全 0 或全 1 数组。empty 可以创建一个没有任何具体值的数组。要用这些方法创建多维数组，只需传入一个表示形状的元组即可：


```
In [26]: np.zeros(10)
Out[26]: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [27]: np.zeros((4,5))
Out[27]:
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [28]: np.empty((2,3,2))
Out[28]:
array([[[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]])


```
注意：认为`np.empty`会返回全 0 数组的想法是不安全的。很多情况下（如前所示），它返回的都是一些未初始化的垃圾值。

- arange 是 Python 内置函数 range 的数组版：

```
In [29]: np.arange(12)
Out[29]: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

```

表 4-1 列出了一些数组创建函数。由于 NumPy 关注的是数值计算，因此，如果没有特别指定，数据类型基本都是 float64（浮点数）。

![表4-1 数组创建函数](./images/numpy.png)


#### ndarray 的数据类型

- dtype（数据类型）是一个特殊的对象，它含有 ndarray 将一块内存解释为特定数据类型所需的信息：

```
In [31]: arr1 = np.array([1,2,3], dtype=np.float64)

In [32]: arr2 = np.array([1,2,3], dtype=np.int32)

In [33]: arr1
Out[33]: array([1., 2., 3.])

In [34]: arr2
Out[34]: array([1, 2, 3])

In [35]: arr1.dtype
Out[35]: dtype('float64')

In [36]: arr2.dtype
Out[36]: dtype('int32')


```

- dtype 是 NumPy 灵活交互其它系统的源泉之一。多数情况下，它们直接映射到相应的机器表示，这使得“读写磁盘上的二进制数据流”以及“集成低级语言代码（如 C、Fortran）”等工作变得更加简单。数值型 dtype 的命名方式相同：一个类型名（如 float 或 int），后面跟一个用于表示各元素位长的数字。标准的双精度浮点值（即 Python 中的 float对象）需要占用 8 字节（即64 位）。因此，该类型在 NumPy 中就记作 float64。表 4-2 列出了 NumPy 所支持的全部数据类型。

- 笔记：记不住这些 NumPy 的 dtype 也没关系，新手更是如此。通常只需要知道你所处理的数据的大致类型是浮点数、复数、整数、布尔值、字符串，还是普通的 Python 对象即可。当你需要控制数据在内存和磁盘中的存储方式时（尤其是对大数据集），那就得了解如何控制存储类型。


![](./images/data_type1.png)
![](./images/data_type2.png)


- 你可以通过 ndarray 的 astype 方法明确地将一个数组从一个 dtype 转换成另一个 dtype：

```
In [37]: arr = np.array([1,2,3,4,5])

In [38]: arr.dtype
Out[38]: dtype('int32')

In [39]: float_arr = arr.astype(np.float64)

In [40]: float_arr.dtype
Out[40]: dtype('float64')


```

- 在本例中，整数被转换成了浮点数。如果将浮点数转换成整数，则小数部分将会被截取删除：

```
In [41]: arr = np.array([3.7, -1.2, -2.6, 0.5, 12.9, 10.1])

In [42]: arr
Out[42]: array([ 3.7, -1.2, -2.6,  0.5, 12.9, 10.1])

In [43]: arr.astype(np.int32)
Out[43]: array([ 3, -1, -2,  0, 12, 10])

```

- 如果某字符串数组表示的全是数字，也可以用 astype 将其转换为数值形式：

```
In [44]: numeric_strings = np.array(['1.25', '9.6', '45'], dtype=np.s
    ...: tring_ )

In [45]: numeric_strings.astype(float)
Out[45]: array([ 1.25,  9.6 , 45.  ])

```

- 注意：使用`numpy.string_`类型时，一定要小心，因为 NumPy 的字符串数据是大小固定的，发生截取时，不会发出警告。pandas 提供了更多非数值数据的便利的处理方法。

- 数组的 dtype 还有另一个属性：

```
In [46]: int_array = np.arange(10)

In [47]: calibers = np.array([.22, .270, .357, .380, .44, .50], dtype=np.float64)

In [48]: int_array.astype(calibers.dtype)
Out[48]: array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

# 你还可以用简洁的类型代码来表示 dtype：

In [49]: empty_uint32 = np.empty(8, dtype='u4')

In [50]: empty_uint32
Out[50]: 
array([         0, 1075314688,          0, 1075707904,          0,
       1075838976,          0, 1072693248], dtype=uint32)

```
- 笔记：调用 astype 总会创建一个新的数组（一个数据的备份），即使新的dtype 与旧的 dtype 相同。

#### NumPy 数组的运算 

数组很重要，因为它使你不用编写循环即可对数据执行批量运算。NumPy 用户称其为矢量化（vectorization）。大小相等的数组之间的任何算术运算都会将运算应用到元素级：

```
In [57]: arr = np.array([[1., 2., 3.], [4., 5., 6.]])

In [58]: arr
Out[58]:
array([[1., 2., 3.],
       [4., 5., 6.]])

In [59]: arr * arr
Out[59]:
array([[ 1.,  4.,  9.],
       [16., 25., 36.]])

In [60]: %time arr - arr
Wall time: 15.6 ms
Out[60]:
array([[0., 0., 0.],
       [0., 0., 0.]])
       
# 数组与标量的算术运算会将标量值传播到各个元素：

In [61]: 1/arr
Out[61]:
array([[1.        , 0.5       , 0.33333333],
       [0.25      , 0.2       , 0.16666667]])

In [62]: arr ** 0.5
Out[62]:
array([[1.        , 1.41421356, 1.73205081],
       [2.        , 2.23606798, 2.44948974]])

# 大小相同的数组之间的比较会生成布尔值数组：

In [63]: arr2 = np.array([[0., 4., 1.], [7., 2., 12.]])

In [64]: arr2
Out[64]:
array([[ 0.,  4.,  1.],
       [ 7.,  2., 12.]])

In [65]: arr2 > arr
Out[65]:
array([[False,  True, False],
       [ True, False,  True]])

```

不同大小的数组之间的运算叫做广播（broadcasting），将在附录A中对其进行详细讨论。本书的内容不需要对广播机制有多深的理解。

#### 基本的索引和切片

NumPy 数组的索引是一个内容丰富的主题，因为选取数据子集或单个元素的方式有很多。一维数组很简单。从表面上看，它们跟 Python 列表的功能差不多:


```
In [60]: arr = np.arange(10)

In [61]: arr
Out[61]: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [62]: arr[5]
Out[62]: 5

In [63]: arr[5:8]
Out[63]: array([5, 6, 7])

In [64]: arr[5:8] = 12

In [65]: arr
Out[65]: array([ 0,  1,  2,  3,  4, 12, 12, 12,  8,  9])

```

如上所示，当你将一个标量值赋值给一个切片时（如 `arr[5:8]=12`），该值会自动传播（也就说后面将会讲到的“广播”）到整个选区。跟列表最重要的区别在于，数组切片是原始数组的视图。这意味着数据不会被复制，视图上的任何修改都会直接反映到源数组上。

```
# 作为例子，先创建一个 arr 的切片：

In [73]: arr_slice = arr[5:8]

In [74]: arr_slice
Out[74]: array([12, 12, 12])

# 现在，当我修稿 arr_slice 中的值，变动也会体现在原始数组 arr 中：

In [75]: arr_slice[1] = 142345

In [76]: arr
Out[76]:
array([     0,      1,      2,      3,      4,     12, 142345,     12,

            8,      9])

# 切片[ : ]会给数组中的所有值赋值：

In [77]: arr_slice[:] = 64

In [78]: arr
Out[78]: array([ 0,  1,  2,  3,  4, 64, 64, 64,  8,  9])


```

- 注意：如果你想要得到的是 ndarray 切片的一份副本而非视图，就需要明确地进行复制操作，例如`arr[5:8].copy()`。


- 对于高维度数组，能做的事情更多。在一个二维数组中，各索引位置上的元素不再是标量而是一维数组：

```
In [79]: arr2d = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [80]: arr2d[2]
Out[80]: array([7, 8, 9])

# 因此，可以对各个元素进行递归访问，但这样需要做的事情有点多。你可以传入一个以逗号隔开的索引列表来选取单个元素。也就是说，下面两种方式是等价的：

In [81]: arr2d[0][2]
Out[81]: 3

In [82]: arr2d[0,2]
Out[82]: 3


```

图4-1 说明了二维数组的索引方式。轴 0 作为行，轴 1 作为列。

![图4-1 NumPy数组中的元素索引](./images/axis.png)

在多维数组中，如果省略了后面的索引，则返回对象会是一个维度低一点的ndarray（它含有高一级维度上的所有数据）。因此，在 2×2×3 数组 arr3d中：

```
In [83]: arr3d = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 1
    ...: 1, 12]]])
    
# arr3d[0] 是一个 2×3 数组：

In [84]: arr3d[0]
Out[84]: 
array([[1, 2, 3],
       [4, 5, 6]])
    
# 标量值和数组都可以被赋值给arr3d[0]：

In [88]: old_values = arr3d[0].copy()

In [89]: arr3d[0] = 42



In [90]: arr3d
Out[90]:
array([[[42, 42, 42],
        [42, 42, 42]],

       [[ 7,  8,  9],
        [10, 11, 12]]])

In [91]: arr3d[0] = old_values

In [92]: arr3d
Out[92]:
array([[[ 1,  2,  3],
        [ 4,  5,  6]],
       [[ 7,  8,  9],
        [10, 11, 12]]])

# 相似的，arr3d[1,0] 可以访问索引以 (1,0) 开头的那些值（以一维数组的形式返回）：

In [100]: arr3d[1,0]
Out[100]: array([7, 8, 9])

# 虽然是用两步进行索引的，表达式是相同的：

In [101]: x = arr3d[1]

In [102]: x
Out[102]:
array([[ 7,  8,  9],
       [10, 11, 12]])

In [103]: x[0]
Out[103]: array([7, 8, 9])

 
```

注意，在上面所有这些选取数组子集的例子中，返回的数组都是视图。

#### 切片索引

```
# ndarray 的切片语法跟 Python 列表这样的一维对象差不多：

In [4]: arr = np.arange(10)

In [5]: arr[1:6]
Out[5]: array([1, 2, 3, 4, 5])

# 对于之前的二维数组 arr2d，其切片方式稍显不同：

In [6]: arr2d = np.arr ay([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [7]: arr3d = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11
   ...: , 12]]])

In [8]: arr2d[:2]
Out[8]:
array([[1, 2, 3],
       [4, 5, 6]])

# 可以看出，它是沿着第 0 轴（即第一个轴）切片的。也就是说，切片是沿着一
# 个轴向选取元素的。表达式 arr2d[:2] 可以被认为是“选取 arr2d 的前两行”。

# 你可以一次传入多个切片，就像传入多个索引那样

In [9]: arr2d[:2,1:]
Out[9]:
array([[2, 3],
       [5, 6]])


# 像这样进行切片时，只能得到相同维数的数组视图。
# 通过将整数索引和切片混合，可以得到低维度的切片。

# 例如，我可以选取第二行的前两列：

In [10]: arr2d[1,:2]
Out[10]: array([4, 5])

# 相似的，还可以选择第三列的前两行：

In [11]: arr2d[:2,2]
Out[11]: array([3, 6])

In [12]: arr2d[:,:1]
Out[12]:
array([[1],
       [4],
       [7]])


```
![](./images/axis_slice.png)

自然，对切片表达式的赋值操作也会被扩散到整个选区：

```
In [96]: arr2d[:2, 1:] = 0

In [97]: arr2d
Out[97]: 
array([[1, 0, 0],
       [4, 0, 0],
       [7, 8, 9]])

```

#### 布尔型索引

来看这样一个例子，假设我们有一个用于存储数据的数组以及一个存储姓名的数组（含有重复项）。在这里，我将使用`numpy.random`中的`randn`函数生成一些正态分布的随机数据：

```
In [13]: names = np.array(['Bob', 'Joe', 'Will', 'Bob', 'Will', 'Joe'
    ...: , 'Joe'])

In [14]: data = np.random.randn(7,4)

In [15]: names
Out[15]: array(['Bob', 'Joe', 'Will', 'Bob', 'Will', 'Joe', 'Joe'], dt
ype='<U4')

In [16]: data
Out[16]:
array([[-0.40297344, -0.04858888,  0.94186535, -0.43067234],
       [ 0.94293832,  0.42824721, -0.47260321, -0.82525473],
       [-1.15078413,  0.12698473,  1.22078788, -0.21719696],
       [-3.39869444, -1.66794053, -0.93024575, -0.77330647],
       [ 0.93072027, -0.53585592, -0.11522692, -1.38992789],
       [-0.37161027, -0.91641934, -0.5625268 , -0.85193452],
       [ 0.42138626,  0.80474264, -0.93333829,  0.54982325]])

# 假设每个名字都对应data数组中的一行，而我们想要选出对应于名字"Bob"的所有行。
# 跟算术运算一样，数组的比较运算（如==）也是矢量化的。
# 因此，对names和字符串"Bob"的比较运算将会产生一个布尔型数组：

In [17]: names == 'Bob'
Out[17]: array([ True, False, False,  True, False, False, False])

# 这个布尔型数组可用于数组索引：

In [18]: data[names=='Bob']
Out[18]:
array([[-0.40297344, -0.04858888,  0.94186535, -0.43067234],
       [-3.39869444, -1.66794053, -0.93024575, -0.77330647]])


```
布尔型数组的长度必须跟被索引的轴长度一致。此外，还可以将布尔型数组跟切片、整数（或整数序列，稍后将对此进行详细讲解）混合使用：


```
In [103]: data[names == 'Bob']
Out[103]: 
array([[ 0.0929,  0.2817,  0.769 ,  1.2464],
       [ 1.669 , -0.4386, -0.5397,  0.477 ]])

```
注意：如果布尔型数组的长度不对，布尔型选择就会出错，因此一定要小心。

```
# 下面的例子，我选取了names == 'Bob'的行，并索引了列：

In [19]: data[names == 'Bob', 2:]
Out[19]:
array([[ 0.94186535, -0.43067234],
       [-0.93024575, -0.77330647]])

In [20]: data[names == 'Bob', 3:]
Out[20]:
array([[-0.43067234],
       [-0.77330647]])

In [21]: data
Out[21]:
array([[-0.40297344, -0.04858888,  0.94186535, -0.43067234],
       [ 0.94293832,  0.42824721, -0.47260321, -0.82525473],
       [-1.15078413,  0.12698473,  1.22078788, -0.21719696],
       [-3.39869444, -1.66794053, -0.93024575, -0.77330647],
       [ 0.93072027, -0.53585592, -0.11522692, -1.38992789],
       [-0.37161027, -0.91641934, -0.5625268 , -0.85193452],
       [ 0.42138626,  0.80474264, -0.93333829,  0.54982325]])


# 要选择除"Bob"以外的其他值，既可以使用不等于符号（!=），也可以通过~对条件进行否定：

In [22]: names != 'Bob'
Out[22]: array([False,  True,  True, False,  True,  True,  True])

In [23]: data[~(names == 'Bob')]
Out[23]:
array([[ 0.94293832,  0.42824721, -0.47260321, -0.82525473],
       [-1.15078413,  0.12698473,  1.22078788, -0.21719696],
       [ 0.93072027, -0.53585592, -0.11522692, -1.38992789],
       [-0.37161027, -0.91641934, -0.5625268 , -0.85193452],
       [ 0.42138626,  0.80474264, -0.93333829,  0.54982325]])

# ~操作符用来反转条件很好用：

In [24]: cond = names == 'Bob'

In [25]: data[~cond]
Out[25]:
array([[ 0.94293832,  0.42824721, -0.47260321, -0.82525473],
       [-1.15078413,  0.12698473,  1.22078788, -0.21719696],
       [ 0.93072027, -0.53585592, -0.11522692, -1.38992789],
       [-0.37161027, -0.91641934, -0.5625268 , -0.85193452],
       [ 0.42138626,  0.80474264, -0.93333829,  0.54982325]])

In [26]: cond
Out[26]: array([ True, False, False,  True, False, False, False])

In [27]: ~cond
Out[27]: array([False,  True,  True, False,  True,  True,  True])


```
选取这三个名字中的两个需要组合应用多个布尔条件，使用 &（和）、|（或）之类的布尔算术运算符即可：

```
In [30]: mask = (names == 'Bob') | (names == 'Will')

In [31]: mask
Out[31]: array([ True, False,  True,  True,  True, False, False])

In [32]: data[mask]
Out[32]:
array([[-0.40297344, -0.04858888,  0.94186535, -0.43067234],
       [-1.15078413,  0.12698473,  1.22078788, -0.21719696],
       [-3.39869444, -1.66794053, -0.93024575, -0.77330647],
       [ 0.93072027, -0.53585592, -0.11522692, -1.38992789]])


```

通过布尔型索引选取数组中的数据，将总是创建数据的副本，即使返回一模一样的数组也是如此。

注意：Python 关键字 and 和 or 在布尔型数组中无效。要使用 & 与 |。

通过布尔型数组设置值是一种经常用到的手段。为了将 data 中的所有负值都设置为 0，我们只需：

```
In [33]: data[data < 0] =0

In [34]: data
Out[34]:
array([[0.        , 0.        , 0.94186535, 0.        ],
       [0.94293832, 0.42824721, 0.        , 0.        ],
       [0.        , 0.12698473, 1.22078788, 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.93072027, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.42138626, 0.80474264, 0.        , 0.54982325]])


```

通过一维布尔数组设置整行或列的值也很简单：


```
In [35]: data[names != 'Joe'] = 7

In [36]: data
Out[36]:
array([[7.        , 7.        , 7.        , 7.        ],
       [0.94293832, 0.42824721, 0.        , 0.        ],
       [7.        , 7.        , 7.        , 7.        ],
       [7.        , 7.        , 7.        , 7.        ],
       [7.        , 7.        , 7.        , 7.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.42138626, 0.80474264, 0.        , 0.54982325]])


```
后面会看到，这类二维数据的操作也可以用 `pandas` 方便的来做。

#### 花式索引


花式索引（Fancy indexing）是一个 NumPy 术语，它指的是利用整数数组进行索引。假设我们有一个8×4数组：

```
In [37]: arr = np.empty((8,4))

In [38]: for i in range(8):
    ...:     arr[i] = i
    ...:

In [39]: arr
Out[39]:
array([[0., 0., 0., 0.],
       [1., 1., 1., 1.],
       [2., 2., 2., 2.],
       [3., 3., 3., 3.],
       [4., 4., 4., 4.],
       [5., 5., 5., 5.],
       [6., 6., 6., 6.],
       [7., 7., 7., 7.]])

# 为了以特定顺序选取行子集，只需传入一个用于指定顺序的整数列表或ndarray 即可：

In [40]: arr[[4,3,0,6]]
Out[40]:
array([[4., 4., 4., 4.],
       [3., 3., 3., 3.],
       [0., 0., 0., 0.],
       [6., 6., 6., 6.]])


# 这段代码确实达到我们的要求了！使用负数索引将会从末尾开始选取行：

In [41]: arr[[-3,-5,-7]]
Out[41]:
array([[5., 5., 5., 5.],
       [3., 3., 3., 3.],
       [1., 1., 1., 1.]])

# 一次传入多个索引数组会有一点特别。它返回的是一个一维数组，其中的元素对应各个索引元组：

In [42]: arr = np.arange(32).reshape((8,4))

In [43]: arr
Out[43]:
array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19],
       [20, 21, 22, 23],
       [24, 25, 26, 27],
       [28, 29, 30, 31]])

In [44]: arr[[1,5,7,2],[0,3,1,2]]
Out[44]: array([ 4, 23, 29, 10])


```

附录A中会详细介绍 reshape 方法。

最终选出的是元素 (1,0)、(5,3)、(7,1)和(2,2)。无论数组是多少维的，花式索引总是一维的。

这个花式索引的行为可能会跟某些用户的预期不一样（包括我在内），选取矩阵的行列子集应该是矩形区域的形式才对。下面是得到该结果的一个办法：

```
In [45]: arr[[1,5,7,2]][:,[0,3,1,2]]
Out[45]:
array([[ 4,  7,  5,  6],
       [20, 23, 21, 22],
       [28, 31, 29, 30],
       [ 8, 11,  9, 10]])

```

- 记住，花式索引跟切片不一样，它总是将数据复制到新数组中。

#### 数组转置和轴对换

转置是重塑的一种特殊形式，它返回的是源数据的视图（不会进行任何复制操作）。数组不仅有`transpose`方法，还有一个特殊的 T 属性：

```


```

```


```

```


```

### 4.2 通用函数：快速的元素级数组函数

```


```

####

### 4.3 利用数组进行数据处理

```


```

#### 将条件逻辑表述为数组运算

```


```

#### 数学和统计方法

```


```

#### 用于布尔型数组的方法

```


```

#### 排序

```


```

#### 唯一化以及其它的集合逻辑

```


```

### 4.4 用于数组的文件输入输出

```


```

### 4.5 线性代数 


```


```

### 4.6 伪随机数生成

```


```

### 4.7 示例：随机漫步

```


```

#### 一次模拟多个随机漫步

```


```

```


```

###  4.8 结论

虽然本书剩下的章节大部分是用 pandas 规整数据，我们还是会用到相似的基于数组的计算。在附录A中，我们会深入挖掘 NumPy 的特点，进一步学习数组的技巧。